# Model monitoring

In [1]:
import os
import glob
import random
import pprint

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyspark

In [2]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

#Pyspark remove warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/09 06:49:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
predictions_directory = "/app/datamart/gold/model_predictions/"
files_list = [predictions_directory+os.path.basename(f) for f in glob.glob(os.path.join(predictions_directory, '*'))]
df_predictions = spark.read.option("header", "true").parquet(*files_list)


In [17]:
df_predictions.show(truncate=False)

+-----------+-------------+-------------------------------+--------------------+
|Customer_ID|snapshot_date|model_name                     |model_predictions   |
+-----------+-------------+-------------------------------+--------------------+
|CUS_0x1d9e |2024-11-01   |xgb_credit_model_2024_09_01.pkl|0.08877021819353104 |
|CUS_0x7ea2 |2024-11-01   |xgb_credit_model_2024_09_01.pkl|0.1391143500804901  |
|CUS_0xedb  |2024-11-01   |xgb_credit_model_2024_09_01.pkl|0.6090103983879089  |
|CUS_0x4b91 |2024-11-01   |xgb_credit_model_2024_09_01.pkl|0.2919446527957916  |
|CUS_0x6b0c |2024-11-01   |xgb_credit_model_2024_09_01.pkl|0.04694114997982979 |
|CUS_0x971a |2024-11-01   |xgb_credit_model_2024_09_01.pkl|0.2924809753894806  |
|CUS_0x6a1a |2024-11-01   |xgb_credit_model_2024_09_01.pkl|0.2607428729534149  |
|CUS_0x7c64 |2024-11-01   |xgb_credit_model_2024_09_01.pkl|0.14412736892700195 |
|CUS_0xab92 |2024-11-01   |xgb_credit_model_2024_09_01.pkl|0.1766100525856018  |
|CUS_0xc16d |2024-11-01   |x

In [16]:
# Load true labels for snapshotdate = 2024_11_01
gold_label_directory = "/app/datamart/gold/label_store/"
files_list = [gold_label_directory+os.path.basename(f) for f in glob.glob(os.path.join(gold_label_directory, '*'))]
label_df = spark.read.option("header", "true").parquet(*files_list)

merged_df = df_predictions.join(
    label_df,
    on=["snapshot_date", "Customer_ID"],
    how="inner"
)

merged_df.show()


+-------------+-----------+--------------------+--------------------+--------------------+-----+----------+
|snapshot_date|Customer_ID|          model_name|   model_predictions|             loan_id|label| label_def|
+-------------+-----------+--------------------+--------------------+--------------------+-----+----------+
|   2024-11-01| CUS_0x102e|xgb_credit_model_...| 0.09399330615997314|CUS_0x102e_2024_0...|    1|90dpd_7mob|
|   2024-11-01| CUS_0x109d|xgb_credit_model_...|  0.5430546402931213|CUS_0x109d_2024_0...|    0|90dpd_7mob|
|   2024-11-01| CUS_0x112e|xgb_credit_model_...| 0.13892267644405365|CUS_0x112e_2024_0...|    0|90dpd_7mob|
|   2024-11-01| CUS_0x1183|xgb_credit_model_...|0.044778548181056976|CUS_0x1183_2024_0...|    0|90dpd_7mob|
|   2024-11-01| CUS_0x1220|xgb_credit_model_...|  0.1285398304462433|CUS_0x1220_2024_0...|    0|90dpd_7mob|
|   2024-11-01| CUS_0x1226|xgb_credit_model_...| 0.10011135041713715|CUS_0x1226_2024_0...|    0|90dpd_7mob|
|   2024-11-01| CUS_0x1343|x

In [ ]:
from pyspark.sql import functions as F

# Replace with the model you want to evaluate
model_to_eval = "xgb_credit_model_2024_09_01"
df_eval = merged_df.filter(F.col("model_name") == model_to_eval)

pdf = df_eval.select("label", "model_predictions").toPandas()

y_true = pdf["label"]
y_pred = pdf["model_predictions"]
y_pred_binary = (y_pred >= 0.5).astype(int)  # adjust threshold if needed


In [ ]:
# assume y_true_pdf and y_pred_pdf are pandas Series aligned by Customer_ID+snapshot_date
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, brier_score_loss

y_true = y_true_pdf.values
y_pred_proba = y_pred_pdf.values  # float probabilities
y_pred_label = (y_pred_proba >= 0.5).astype(int)  # threshold, tune per business

metrics = {
    "n": int(len(y_true)),
    "auc": float(roc_auc_score(y_true, y_pred_proba)),
    "precision": float(precision_score(y_true, y_pred_label, zero_division=0)),
    "recall": float(recall_score(y_true, y_pred_label, zero_division=0)),
    "f1": float(f1_score(y_true, y_pred_label, zero_division=0)),
    "brier": float(brier_score_loss(y_true, y_pred_proba)),
    "positive_rate": float(y_pred_label.mean()),
    "pred_mean": float(y_pred_proba.mean()),
}
